In [ ]:
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from groq import Groq
import json

client_groq = Groq(api_key="YOUR_GROQ_KEY")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def cosine_sim(a, b):
    return np.dot(a,b) / (np.linalg.norm(a)*np.linalg.norm(b))

def chat_with_rag(query):
    # fetch docs from Laravel
    docs = requests.get("http://127.0.0.1:8000/api/docs").json()

    qvec = embed_model.encode(query)

    # compute cosine similarity in Python
    scored = []
    for d in docs:
        emb = np.array(json.loads(d["embedding"]))
        score = cosine_sim(qvec, emb)
        scored.append((score, d))

    # top 3 results
    top = sorted(scored, key=lambda x: x[0], reverse=True)[:3]

    # build context
    context = "\n\n".join(d["text"] for _, d in top)

    messages = [
        {"role": "system", "content": "Use only the context below:"},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
    ]

    resp = client_groq.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        max_tokens=400,
        temperature=0
    )
    return resp.choices[0].message.content
    
    

print(chat_with_rag("Laravel opening available?"))


There is no mention of a Laravel opening in the provided context. The context only mentions an opportunity to work with Generative AI models, but it does not specify the technology or framework (such as Laravel) being used.
